In [109]:
#combining all datasets using map function

import glob
import pandas as pd 

#map renewables

df_renew = pd.concat(map(pd.read_csv, glob.glob('C:/Users/Rachel/OneDrive/Documents/Springboard/CAPSTONE 2/energy/raw_data_renewable/*.csv')))
df_renew.rename(columns={'Country or Area':'Geo'}, inplace=True)

#map nonrenewables

df_nonrenew = pd.concat(map(pd.read_csv, glob.glob('C:/Users/Rachel/OneDrive/Documents/Springboard/CAPSTONE 2/energy/raw_data_nonrenewable/*.csv')))
df_nonrenew.rename(columns={'Country or Area':'Geo'}, inplace=True)

df = pd.concat([df_renew, df_nonrenew])

#split commodity and transaction into two columns
#defining column to split and which delimiter
ct = df['Commodity - Transaction'].str.split('-')

#assigning new columns
df['Commodity'] = ct.str.get(0)
df['Transaction'] = ct.str.get(1)

#lowercase and remove trailing and leading spaces
df['Commodity'] = df['Commodity'].str.lower()
df['Commodity'] = df['Commodity'].str.strip()

df['Transaction'] = df['Transaction'].str.lower()
df['Transaction'] = df['Transaction'].str.strip()

df = df[df['Transaction'] == 'production']
df.fillna(value=1, method=None, axis=None, inplace=True, limit=None, downcast=None)


In [110]:
df_filt = df[df['Geo'] == 'Monaco']

df_filt

,Geo,Commodity - Transaction,Year,Unit,Quantity,Quantity Footnotes,Commodity,Transaction


In [111]:
import pandas as pd 
unit_conv = pd.read_csv('C:/Users/Rachel/OneDrive/Documents/Springboard/CAPSTONE 2/energy/renewable_energy_units_conv_2.csv')

df_merge = pd.merge(df,  
                     unit_conv,  
                     on ='Commodity', 
                     how ='left') 

df_merge['TJ'] = df_merge['Quantity'] * df_merge['Conv_mt_to_tj']
df_merge.sample(3)

,Geo,Commodity - Transaction,Year,Unit,Quantity,Quantity Footnotes,Commodity,Transaction,Description,Units,Conv_mt_to_tj,r_type,TJ
17391,Croatia,Hydro electricity-production,2015.0,"Kilowatt-hours, million",6556.0,1.0,hydro electricity,production,NaN,"Kilowatt-hours, million",3.57,renewable,23404.92
42921,United Kingdom,Ethane - Production,2011.0,"Metric tons, thousand",834.0,1.0,ethane,production,Isolated from Natural Gas,"Metric tons, thousand",46.40,nonrenewable,38697.60
52171,Bolivia (Plur. State of),Liquefied petroleum gas (LPG) - Production,1990.0,"Metric tons, thousand",194.0,1.0,liquefied petroleum gas (lpg),production,NaN,"Metric tons, thousand",47.30,nonrenewable,9176.20


In [112]:

#sum all energy types, df_merge is the combination of unit conversion table and raw data to sum total TJ
#filter by year
lt_2018 = df_merge[df_merge['Year'] == 2018]
#greate geo group
tot_grp_country_2018 = lt_2018.groupby(['Geo'])
#sum TJ by geo group
df_tot_country_2018 = tot_grp_country_2018['TJ'].agg(['sum'])

#sum all renewables
lt_2018_r = df_merge[(df_merge['Year'] == 2018) & (df_merge['r_type'] == 'renewable')]
tot_r_grp_country_2018 = lt_2018_r.groupby(['Geo'])
df_tot_country_r_2018 = tot_r_grp_country_2018['TJ'].agg(['sum'])

#sum all nonrenewables
lt_2018_n = df_merge[(df_merge['Year'] == 2018) & (df_merge['r_type'] == 'nonrenewable')]
tot_n_grp_country_2018 = lt_2018_n.groupby(['Geo'])
df_tot_country_n_2018 = tot_n_grp_country_2018['TJ'].agg(['sum'])

In [113]:
#merge total, renewable, and nonrenewable dataframes

countries_r_2018 = pd.merge(df_tot_country_2018,  
                     df_tot_country_r_2018,  
                     on ='Geo', 
                     how ='left') 

countries_r_n_2018 = pd.merge(countries_r_2018,
                    df_tot_country_n_2018,
                    on = 'Geo', 
                    how = 'left')

countries_r_n_2018.rename(columns={'sum_x':'total', 'sum_y': 'renew', 'sum' : 'nonrenew'}, inplace=True)
countries_r_n_2018['percent_renewable'] = (countries_r_n_2018['renew'] / countries_r_n_2018['total'])
countries_r_n_2018.sort_values(by='percent_renewable', ascending=False, inplace=True)
#allsources_r_n['diff'] = allsources_r_n['renew'] + allsources_r_n['nonrenew']#




# countries_r_n_2018.to_csv('C:/Users/Rachel/OneDrive/Documents/Springboard/CAPSTONE 2/energy/presentation links/perc_renewable_by_country_2018.csv')

In [114]:
#sum all energy types, df_merge is the combination of unit conversion table and raw data to sum total TJ
#filter by year
lt_2008 = df_merge[df_merge['Year'] == 2008]
#greate geo group
tot_grp_country_2008 = lt_2008.groupby(['Geo'])
#sum TJ by geo group
df_tot_country_2008 = tot_grp_country_2008['TJ'].agg(['sum'])

#sum all renewables
lt_2008_r = df_merge[(df_merge['Year'] == 2008) & (df_merge['r_type'] == 'renewable')]
tot_r_grp_country_2008 = lt_2008_r.groupby(['Geo'])
df_tot_country_r_2008 = tot_r_grp_country_2008['TJ'].agg(['sum'])

#sum all nonrenewables
lt_2008_n = df_merge[(df_merge['Year'] == 2008) & (df_merge['r_type'] == 'nonrenewable')]
tot_n_grp_country_2008 = lt_2008_n.groupby(['Geo'])
df_tot_country_n_2008 = tot_n_grp_country_2008['TJ'].agg(['sum'])

In [115]:
#merge total, renewable, and nonrenewable dataframes

countries_r_2008 = pd.merge(df_tot_country_2008,  
                     df_tot_country_r_2008,  
                     on ='Geo', 
                     how ='left') 

countries_r_n_2008 = pd.merge(countries_r_2008,
                    df_tot_country_n_2008,
                    on = 'Geo', 
                    how = 'left')

countries_r_n_2008.rename(columns={'sum_x':'total', 'sum_y': 'renew', 'sum' : 'nonrenew'}, inplace=True)
countries_r_n_2008['percent_renewable'] = ((countries_r_n_2008['renew'] / countries_r_n_2008['total']))
countries_r_n_2008.sort_values(by='percent_renewable', ascending=False, inplace=True)
countries_r_n_2008.reset_index(inplace=True)
countries_r_n_2008.fillna(value=0, inplace=True)
#allsources_r_n['diff'] = allsources_r_n['renew'] + allsources_r_n['nonrenew']#
# countries_r_n_2008.to_csv('C:/Users/Rachel/OneDrive/Documents/Springboard/CAPSTONE 2/energy/presentation links/perc_renewable_by_country_2008.csv')

countries_r_n_2008.loc[countries_r_n_2008['Geo'] == 'Iceland']

,Geo,total,renew,nonrenew,percent_renewable
21,Iceland,58949.05,58949.05,0.0,1.0


In [116]:
# countries_r_n_2008.drop(columns=['Geo'], inplace=True)
# countries_r_n_2018.drop(columns=['Geo'], inplace=True)
countries_pc_renew = pd.merge(countries_r_n_2008,  
                     countries_r_n_2018,  
                     on = 'Geo', 
                     how ='left') 

countries_pc_renew.drop(columns = ['total_x', 'renew_x', 'nonrenew_x', 'total_y', 'renew_y', 'nonrenew_y'], inplace=True)
countries_pc_renew.rename(columns = {'percent_renewable_x':'pr 2008', 'percent_renewable_y' : 'pr 2018'}, inplace=True)
countries_pc_renew['p_change'] = (countries_pc_renew['pr 2018'] - (countries_pc_renew['pr 2008'])) / countries_pc_renew['pr 2008']
countries_pc_renew.sort_values('p_change', ascending=False, inplace=True)
# countries_pc_renew = countries_pc_renew[countries_pc_renew['p_change'] < 11]
countries_pc_renew.reset_index(inplace=True)
countries_pc_renew['pr 2008'] = countries_pc_renew['pr 2008']
countries_pc_renew.loc[countries_pc_renew['Geo'] == 'Iceland']

# countries_pc_renew.to_csv('C:/Users/Rachel/OneDrive/Documents/Springboard/CAPSTONE 2/energy/presentation links/p_change_by_country.csv')


,index,Geo,pr 2008,pr 2018,p_change
141,21,Iceland,1.0,1.0,0.0


In [128]:
df_merge.head()

,Geo,Commodity - Transaction,Year,Unit,Quantity,Quantity Footnotes,Commodity,Transaction,Description,Units,Conv_mt_to_tj,r_type,TJ
0,Armenia,Animal waste - Production,2018.0,Terajoules,1289.328,1.0,animal waste,production,NaN,Terajoules,1.0,renewable,1289.328
1,Armenia,Animal waste - Production,2017.0,Terajoules,2427.783,1.0,animal waste,production,NaN,Terajoules,1.0,renewable,2427.783
2,Armenia,Animal waste - Production,2016.0,Terajoules,2195.136,1.0,animal waste,production,NaN,Terajoules,1.0,renewable,2195.136
3,Armenia,Animal waste - Production,2015.0,Terajoules,1307.467,1.0,animal waste,production,NaN,Terajoules,1.0,renewable,1307.467
4,Armenia,Animal waste - Production,2014.0,Terajoules,82.000,1.0,animal waste,production,NaN,Terajoules,1.0,renewable,82.000


In [135]:
#to get the breakdown of types for the top 10 'natural' group

import pandas as pd
import glob

#sum all energy types, df_merge is the combination of unit conversion table and raw data to sum total TJ
#filter by year
for geo in df_merge['Geo'].unique() : 
    country_types = df_merge[(df_merge['Year'] == 2018) & (df_merge['Geo'] == geo)]
    #greate geo group
    country_type_grp = country_types.groupby(['Geo','Commodity'])
    #sum TJ by geo group
    country_types = country_type_grp['TJ'].agg(['sum'])
    country_types.sort_values(by='sum', ascending=False, inplace=True)



    #comment out unless you want to export 200+ csv files:
    country_types.to_csv('C:/Users/Rachel/OneDrive/Documents/Springboard/CAPSTONE 2/energy/renewable_types_by_country/' + geo + '.csv')

In [108]:
#export results to csv for tableau viz, keep commented out

#countries_r_n.to_csv('countries_percent_renewable_2013.csv')